In [1]:
import pandas as pd
import numpy as np

# Create sample dataset
sample_data = pd.DataFrame({
    'id': range(1, 10001),
    'date': pd.date_range(start='2020-01-01', periods=10000, freq='H'),
    'ticker': np.random.choice(['AAPL', 'GOOGL', 'AMZN', 'MSFT'], size=10000),
    'close': np.random.uniform(100, 1000, size=10000)
})
sample_data.to_csv('../data/sample_data.csv', index=False)


C:\Users\Sachin Gora\AppData\Local\Temp\ipykernel_22176\4223550132.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'date': pd.date_range(start='2020-01-01', periods=10000, freq='H'),


In [2]:
#Measure time to insert data into MySQL:

import time
import mysql.connector

# Connect to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Root",
    database="stock_analysis"
)
cursor = connection.cursor()

# Load sample data
sample_data = pd.read_csv('../data/sample_data.csv')

# Measure insertion time
start_time = time.time()
for _, row in sample_data.iterrows():
    cursor.execute("""
    INSERT INTO stock_prices (date, ticker, close)
    VALUES (%s, %s, %s)
    """, (row['date'], row['ticker'], row['close']))
connection.commit()
end_time = time.time()
print(f"MySQL Insertion Time: {end_time - start_time} seconds")


MySQL Insertion Time: 4.256014585494995 seconds


In [3]:
#Measure time to insert data into MongoDB

from pymongo import MongoClient
import time

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['stock_analysis']

# Load sample data
sample_data_dict = sample_data.to_dict('records')

# Measure insertion time
start_time = time.time()
db.stock_prices.insert_many(sample_data_dict)
end_time = time.time()
print(f"MongoDB Insertion Time: {end_time - start_time} seconds")


MongoDB Insertion Time: 0.15366864204406738 seconds


In [4]:
#Measure time for a query (e.g., average close price for AAPL):

query = """
SELECT AVG(close) FROM stock_prices WHERE ticker = 'AAPL'
"""
start_time = time.time()
cursor.execute(query)
result = cursor.fetchone()
end_time = time.time()
print(f"MySQL Query Time: {end_time - start_time} seconds, Result: {result}")


MySQL Query Time: 0.02513432502746582 seconds, Result: (475.9381716104473,)


In [5]:
#Measure time for a similar query in MongoDB

start_time = time.time()
result = db.stock_prices.aggregate([
    {"$match": {"ticker": "AAPL"}},
    {"$group": {"_id": None, "avg_close": {"$avg": "$close"}}}
])
end_time = time.time()
print(f"MongoDB Query Time: {end_time - start_time} seconds, Result: {list(result)}")


MongoDB Query Time: 0.09668684005737305 seconds, Result: [{'_id': None, 'avg_close': 552.4653363254581}]


In [ ]:
comparison = pd.DataFrame({
    "Metric": ["Data Ingestion Time", "Query Execution Time"],
    "MySQL": [4.256014585494995, 0.02513432502746582],
    "MongoDB": [0.15366864204406738, 0.09668684005737305]
})
comparison.to_csv('../data/database_comparison.csv', index=False)
